This is just the copy of the orginal file. With additional calculations.
Difference: Implementation of THR and p-values.

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchcp.classification.score import APS, RAPS
from torchcp.classification.predictor import SplitPredictor
import torch.nn.functional as F
from tqdm import tqdm
from torchcp.classification.score import THR
from torchcp.classification.predictor import SplitPredictor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [12]:
def create_advanced_model(num_classes):
    model = models.resnet18(pretrained=True)
    in_features = model.fc.in_features
    
    model.fc = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(in_features, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )
    return model

In [ ]:
#Loading the model
model = create_advanced_model(num_classes=3)
model.load_state_dict(torch.load("best_covid_model.pth"))
model.eval()
model = model.to(device)
print("Model loaded successfully!")

c:\Users\User\anaconda3\envs\dissertation\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\envs\dissertation\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded successfully!


In [ ]:
# Copying the same code from the training script to avoid to avoid import errors
import os
from glob import glob
from pathlib import Path
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms

# Dataset paths
BASE_PATH = r"C:\Users\User\Desktop\Dissertation\Code\dataset2\COVID-19_Radiography_Dataset"
COVID_PATH = os.path.join(BASE_PATH, "COVID", "images")
NORMAL_PATH = os.path.join(BASE_PATH, "Normal", "images")
VIRAL_PNEUMONIA_PATH = os.path.join(BASE_PATH, "Viral Pneumonia", "images")

def load_image_paths():
    covid_images = glob(os.path.join(COVID_PATH, "*.png")) 
    normal_images = glob(os.path.join(NORMAL_PATH, "*.png"))
    viral_pneumonia_images = glob(os.path.join(VIRAL_PNEUMONIA_PATH, "*.png"))
    return covid_images, normal_images, viral_pneumonia_images

def get_eval_transform():
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

class CovidDataset(Dataset):
    def __init__(self, df, transform, class_roots):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.class_roots = class_roots

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        label = int(row['label'])
        img_name = row['Image Index']
        
        img_path = os.path.join(self.class_roots[label], img_name)
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        
        return img, torch.tensor(label, dtype=torch.long)

def paths_to_df(paths, labels):
    return pd.DataFrame({
        'Image Index': [Path(p).name for p in paths],
        'label': labels
    })

# Recreate the same data split and loaders
covid_paths, normal_paths, viral_pneumonia_paths = load_image_paths()

normal_labels = [0] * len(normal_paths)
covid_labels = [1] * len(covid_paths)
viral_pneumonia_labels = [2] * len(viral_pneumonia_paths)

all_images = normal_paths + covid_paths + viral_pneumonia_paths
all_labels = normal_labels + covid_labels + viral_pneumonia_labels

# Same split as in dataset.py (important: same random_state!)
X_temp, X_test, y_temp, y_test = train_test_split(
    all_images, all_labels, test_size=0.25, random_state=42, stratify=all_labels)

X_train, X_cal, y_train, y_cal = train_test_split(
    X_temp, y_temp, test_size=0.333, random_state=42, stratify=y_temp)

eval_transform = get_eval_transform()

class_roots = {
    0: os.path.join(BASE_PATH, "Normal", "images"),
    1: os.path.join(BASE_PATH, "COVID", "images"),
    2: os.path.join(BASE_PATH, "Viral Pneumonia", "images")
}


cal_df = paths_to_df(X_cal, y_cal)
test_df = paths_to_df(X_test, y_test)

cal_dataset = CovidDataset(cal_df, eval_transform, class_roots)
test_dataset = CovidDataset(test_df, eval_transform, class_roots)

cal_loader = DataLoader(cal_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)





DataLoaders recreated successfully!


In [5]:
def get_predictions(model, dataloader):
    model.eval()
    all_logits = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Getting predictions"):
            images = images.to(device)
            outputs = model(images)
            all_logits.append(outputs.cpu())
            all_labels.append(labels)
    
    return torch.cat(all_logits), torch.cat(all_labels)

In [ ]:
#extracting and saving predicitions
cal_logits, cal_labels = get_predictions(model, cal_loader)
test_logits, test_labels = get_predictions(model, test_loader)

# Save predictions
torch.save({
    'cal_logits': cal_logits, 'cal_labels': cal_labels,
    'test_logits': test_logits, 'test_labels': test_labels
}, 'covid_model_logits.pt')

Getting predictions: 100%|██████████| 119/119 [08:22<00:00,  4.23s/it]


now we are going to try and get the results with label ranking.

In [ ]:

data = torch.load('covid_model_logits.pt')
cal_logits = data['cal_logits']
cal_labels = data['cal_labels']
test_logits = data['test_logits']
test_labels = data['test_labels']

print(f"Loaded predictions - Cal: {cal_logits.shape}, Test: {test_logits.shape}")

Loaded predictions - Cal: torch.Size([3785, 3]), Test: torch.Size([3789, 3])


In [ ]:
from torchcp.classification.score import THR
from torchcp.classification.predictor import SplitPredictor


naive_score = THR(score_type="softmax")
naive_predictor = SplitPredictor(score_function=naive_score)

naive_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)


naive_prediction_sets = naive_predictor.predict_with_logits(test_logits)


print("=== Naive Softmax Results ===")
print(f"Average set size: {naive_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {naive_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")

=== Naive Softmax Results ===
Average set size: 0.91
Coverage: 0.897


In [ ]:
from torchcp.classification.score import THR
from torchcp.classification.predictor import SplitPredictor

thr_score = THR(score_type="softmax")
thr_predictor = SplitPredictor(score_function=thr_score)


alphas = [0.05, 0.1, 0.15, 0.2]

print("=== THR (Naive Softmax) Results Across Alpha ===")
for alpha in alphas:
    # Create fresh predictor for each alpha
    thr_score = THR(score_type="softmax")
    thr_predictor = SplitPredictor(score_function=thr_score)
    
    # Calibrate and predict
    thr_predictor.calculate_threshold(cal_logits, cal_labels, alpha=alpha)
    thr_prediction_sets = thr_predictor.predict_with_logits(test_logits)
    
    avg_size = thr_prediction_sets.sum(dim=1).float().mean().item()
    coverage = thr_prediction_sets[range(len(test_labels)), test_labels].float().mean().item()
    
    print(f"Alpha: {alpha:.2f} | Coverage: {coverage:.3f} | Avg Set Size: {avg_size:.2f}")

=== THR (Naive Softmax) Results Across Alpha ===
Alpha: 0.05 | Coverage: 0.949 | Avg Set Size: 0.96
Alpha: 0.10 | Coverage: 0.897 | Avg Set Size: 0.91
Alpha: 0.15 | Coverage: 0.838 | Avg Set Size: 0.84
Alpha: 0.20 | Coverage: 0.791 | Avg Set Size: 0.79


In [7]:
# Conformal prediction with APS
from torchcp.classification.score import APS
from torchcp.classification.predictor import SplitPredictor

print("Setting up APS conformal predictor...")

Setting up APS conformal predictor...


In [ ]:

aps_score = APS(score_type="softmax", randomized=False)#this one is using false
aps_predictor = SplitPredictor(score_function=aps_score)

# Calculate threshold with calibration logits
aps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)

# Generate prediction sets for test data
aps_prediction_sets = aps_predictor.predict_with_logits(test_logits)

# Show results
print("=== APS Results ===")
print(f"Average set size: {aps_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {aps_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")

=== APS Results ===
Average set size: 1.52
Coverage: 0.907


In [ ]:
# Create APS predictor
aps_score = APS(score_type="softmax")#this one use true
aps_predictor = SplitPredictor(score_function=aps_score)

# Calculate threshold with calibration logits
aps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)

# Generate prediction sets for test data
aps_prediction_sets = aps_predictor.predict_with_logits(test_logits)

# Show results
print("=== APS Results ===")
print(f"Average set size: {aps_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {aps_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")

=== APS Results ===
Average set size: 0.94
Coverage: 0.893


In [ ]:

def calculate_aps_pvalues_from_predictor(aps_predictor, cal_logits, cal_labels, test_logits):
    """
    P-value is equal to P(calibration score > candidate label's score)
    """
    # Calculate calibration scores using the same score function
    cal_probs = F.softmax(cal_logits, dim=1)
    cal_scores = aps_predictor.score_function._calculate_single_label(cal_probs, cal_labels)

    test_probs = F.softmax(test_logits, dim=1)

    all_pvalues = []
    
    for i in range(len(test_logits)):
        sample_pvalues = []
        
        # For each possible class
        for class_idx in range(test_probs.shape[1]):
            # Calculate APS score if this class were the true class
            candidate_labels = torch.tensor([class_idx])
            candidate_score = aps_predictor.score_function._calculate_single_label(test_probs[i:i+1], candidate_labels)
            
            # P-value = proportion of calibration scores > candidate score
            pvalue = (cal_scores > candidate_score).float().mean().item()
            sample_pvalues.append(pvalue)
        
        all_pvalues.append(sample_pvalues)
    
    return torch.tensor(all_pvalues)

In [ ]:
def display_ranked_predictions(test_logits, test_labels, pvalues, alpha=0.1, sample_idx=0):
    """
    Display predictions ranked by p-values for a single sample
    """
    class_names = ['Normal', 'COVID-19', 'Viral Pneumonia']
    
    sample_pvalues = pvalues[sample_idx]
    true_label = test_labels[sample_idx].item()
    
    # Creating a ranked list
    ranked_results = []
    for class_idx in range(len(sample_pvalues)):
        pval = sample_pvalues[class_idx].item()
        included = "✓" if pval >= alpha else "✗"
        ranked_results.append((class_idx, class_names[class_idx], pval, included))
    
    # Sort by p-value
    ranked_results.sort(key=lambda x: x[2], reverse=True)
    
    print(f"\n=== Sample {sample_idx} (True label: {class_names[true_label]}) ===")
    print("Rank | Class           | P-value | Included")
    print("-" * 45)
    for rank, (class_idx, class_name, pval, included) in enumerate(ranked_results, 1):
        print(f"{rank:4d} | {class_name:15s} | {pval:7.3f} | {included:8s}")

pvalues = calculate_aps_pvalues_from_predictor(aps_predictor, cal_logits, cal_labels, test_logits)
for i in range(20):
    display_ranked_predictions(test_logits, test_labels, pvalues, alpha=0.1, sample_idx=i)


=== Sample 0 (True label: COVID-19) ===
Rank | Class           | P-value | Included
---------------------------------------------
   1 | COVID-19        |   0.474 | ✓       
   2 | Normal          |   0.002 | ✗       
   3 | Viral Pneumonia |   0.001 | ✗       

=== Sample 1 (True label: COVID-19) ===
Rank | Class           | P-value | Included
---------------------------------------------
   1 | COVID-19        |   0.516 | ✓       
   2 | Viral Pneumonia |   0.022 | ✗       
   3 | Normal          |   0.004 | ✗       

=== Sample 2 (True label: Normal) ===
Rank | Class           | P-value | Included
---------------------------------------------
   1 | Normal          |   0.900 | ✓       
   2 | COVID-19        |   0.008 | ✗       
   3 | Viral Pneumonia |   0.000 | ✗       

=== Sample 3 (True label: Normal) ===
Rank | Class           | P-value | Included
---------------------------------------------
   1 | Normal          |   0.706 | ✓       
   2 | COVID-19        |   0.000 | ✗    

In [ ]:
aps_score = APS(score_type="softmax", randomized=False)
aps_predictor = SplitPredictor(score_function=aps_score)

aps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)


aps_prediction_sets = aps_predictor.predict_with_logits(test_logits)


print("=== APS Results ===")
print(f"Average set size: {aps_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {aps_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")

In [48]:
alphas = [0.05, 0.1, 0.15, 0.2]

print("=== APS Results Across Alpha ===")
for alpha in alphas:
    aps_score = APS(score_type="softmax", randomized=True)
    aps_predictor = SplitPredictor(score_function=aps_score)

    aps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=alpha)
    aps_prediction_sets = aps_predictor.predict_with_logits(test_logits)

    avg_size = aps_prediction_sets.sum(dim=1).float().mean().item()
    coverage = aps_prediction_sets[range(len(test_labels)), test_labels].float().mean().item()

    print(f"Alpha: {alpha:.2f} | Coverage: {coverage:.3f} | Avg Set Size: {avg_size:.2f}")

=== APS Results Across Alpha ===
Alpha: 0.05 | Coverage: 0.952 | Avg Set Size: 1.03
Alpha: 0.10 | Coverage: 0.903 | Avg Set Size: 0.95
Alpha: 0.15 | Coverage: 0.835 | Avg Set Size: 0.87
Alpha: 0.20 | Coverage: 0.812 | Avg Set Size: 0.85


In [ ]:
alphas = [0.05, 0.1, 0.15, 0.2]

print("=== RAPS Results Across Alpha ===")
for alpha in alphas:
    raps_score = RAPS(score_type="softmax", randomized=True, penalty=5, kreg=1)
    raps_predictor = SplitPredictor(score_function=raps_score)

    raps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=alpha)
    raps_prediction_sets = raps_predictor.predict_with_logits(test_logits)

    avg_size = raps_prediction_sets.sum(dim=1).float().mean().item()
    coverage = raps_prediction_sets[range(len(test_labels)), test_labels].float().mean().item()

    print(f"Alpha: {alpha:.2f} | Coverage: {coverage:.3f} | Avg Set Size: {avg_size:.2f}")

=== RAPS Results Across Alpha ===
Alpha: 0.05 | Coverage: 0.959 | Avg Set Size: 0.98
Alpha: 0.10 | Coverage: 0.924 | Avg Set Size: 0.95
Alpha: 0.15 | Coverage: 0.860 | Avg Set Size: 0.88
Alpha: 0.20 | Coverage: 0.807 | Avg Set Size: 0.83


In [ ]:
from torchcp.classification.score import RAPS
from torchcp.classification.predictor import SplitPredictor

# Usingdifferent lambda (penalty) values
penalty_values = [0, 0.1, 0.5, 1, 5]
kreg = 1  

print("=== RAPS Results Across Lambda (Penalty) ===")
for penalty in penalty_values:
    raps_score = RAPS(score_type="softmax", randomized=True, penalty=penalty, kreg=kreg)
    raps_predictor = SplitPredictor(score_function=raps_score)

    # Calibrate and predict
    raps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)
    raps_prediction_sets = raps_predictor.predict_with_logits(test_logits)

    # Evaluate
    coverage = raps_prediction_sets[range(len(test_labels)), test_labels].float().mean().item()
    avg_size = raps_prediction_sets.sum(dim=1).float().mean().item()

    print(f"Lambda: {penalty:<4} | Coverage: {coverage:.3f} | Avg Set Size: {avg_size:.2f}")


=== RAPS Results Across Lambda (Penalty) ===
Lambda: 0    | Coverage: 0.899 | Avg Set Size: 0.95
Lambda: 0.1  | Coverage: 0.909 | Avg Set Size: 0.96
Lambda: 0.5  | Coverage: 0.902 | Avg Set Size: 0.95
Lambda: 1    | Coverage: 0.908 | Avg Set Size: 0.96
Lambda: 5    | Coverage: 0.892 | Avg Set Size: 0.94


In [ ]:
from torchcp.classification.score import RAPS
from torchcp.classification.predictor import SplitPredictor

# Fixed lambda value
lambda_val = 0.1
kreg_values = [0, 1, 2, 3]

print(f"=== RAPS Results for λ = {lambda_val} with varying k_reg ===")
for k in kreg_values:
    raps_score = RAPS(score_type="softmax", randomized=True, penalty=lambda_val, kreg=k)
    raps_predictor = SplitPredictor(score_function=raps_score)
    
    # Calibrate
    raps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)
    
    # Predict
    raps_prediction_sets = raps_predictor.predict_with_logits(test_logits)
    
    avg_size = raps_prediction_sets.sum(dim=1).float().mean().item()
    coverage = raps_prediction_sets[range(len(test_labels)), test_labels].float().mean().item()
    
    print(f"k_reg = {k}: Average Set Size = {avg_size:.2f}, Coverage = {coverage:.3f}")


=== RAPS Results for λ = 0.1 with varying k_reg ===
k_reg = 0: Average Set Size = 0.93, Coverage = 0.897
k_reg = 1: Average Set Size = 0.93, Coverage = 0.896
k_reg = 2: Average Set Size = 0.95, Coverage = 0.903
k_reg = 3: Average Set Size = 0.97, Coverage = 0.913


In [43]:
# Create RAPS predictor
raps_score = RAPS(score_type="softmax", randomized=False, penalty=1, kreg=1)
raps_predictor = SplitPredictor(score_function=raps_score)

# Calculate threshold with calibration logits
raps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.1)

# Generate prediction sets for test data
raps_prediction_sets = raps_predictor.predict_with_logits(test_logits)

# Show results
print("=== RAPS Results ===")
print(f"Average set size: {raps_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {raps_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")


=== RAPS Results ===
Average set size: 1.00
Coverage: 0.974


=== RAPS Results Across Alpha ===
Alpha: 0.05 | Coverage: 0.956 | Avg Set Size: 0.98
Alpha: 0.10 | Coverage: 0.919 | Avg Set Size: 0.95
Alpha: 0.15 | Coverage: 0.873 | Avg Set Size: 0.90
Alpha: 0.20 | Coverage: 0.832 | Avg Set Size: 0.86


In [46]:
# Create RAPS predictor
raps_score = RAPS(score_type="softmax", randomized=True, penalty=1, kreg=1)
raps_predictor = SplitPredictor(score_function=raps_score)

# Calculate threshold with calibration logits
raps_predictor.calculate_threshold(cal_logits, cal_labels, alpha=0.05)

# Generate prediction sets for test data
raps_prediction_sets = raps_predictor.predict_with_logits(test_logits)

# Show results
print("=== RAPS Results ===")
print(f"Average set size: {raps_prediction_sets.sum(dim=1).float().mean():.2f}")
print(f"Coverage: {raps_prediction_sets[range(len(test_labels)), test_labels].float().mean():.3f}")


=== RAPS Results ===
Average set size: 0.98
Coverage: 0.953
